## Training softmax model on train data : For standalone probabilities

* This notebook will add some more feautres we have considered later : such as length and adding prev and next 
  context means : feature of prev word and next word ll be appended to current word feature 
* Then we ll at last run softmax model for multiclass classfication and train it on training dataset

In [1]:
# importing necessary packages
import pandas as pd
import numpy as np

### loading featurized data matrix that we have already stored in disk (final_df.csv)

In [2]:
df = pd.read_csv('./final_df.csv')
df.head()

,word,tag,capitalize,digit_first,s_dig__e_alpha,p_anti,p_pre,p_un,p_dis,p_inter,...,s_ed_ing,s_tion_ion,s_est,s_less,s_e_es,s_en,s_ly,s_er,s_'s_s',prev_word
0,Confidence,NN,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,in,IN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,the,DT,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,pound,NN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,is,VBZ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Adding some more features : length , hyphen , all capital etc.

In [3]:
length = []
for s in df['word']:
    length.append(len(s))

length_2 = np.asarray(length).reshape(-1,1)
df['length'] = length_2
df.head()

,word,tag,capitalize,digit_first,s_dig__e_alpha,p_anti,p_pre,p_un,p_dis,p_inter,...,s_tion_ion,s_est,s_less,s_e_es,s_en,s_ly,s_er,s_'s_s',prev_word,length
0,Confidence,NN,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,10
1,in,IN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,the,DT,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3
3,pound,NN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
4,is,VBZ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [48]:
hyphen = []
all_cap = []
sp_char = []
cap_dot = []
number = []
st_alpha_dot = []
e_day = []
for i in df['word']:
    if('-' in i):
        hyphen.append(1)
    else:
        hyphen.append(0)
    if(i.isupper()):
        all_cap.append(1)
    else:
        all_cap.append(0)
    if(('$' in i or ',' in i or '%' in i or '!' in i or '\'' in i or '"' in i) \
       and len(i) == 1):
        sp_char.append(1)
    else:
        sp_char.append(0)
    if(i[0].isupper() and i.endswith('.')):
        cap_dot.append(1)
    else:
        cap_dot.append(0)
    if(i[0].isdigit() and i[-1].isdigit() and ('.' in i or ',' in i)):
        number.append(1)
    else:
        number.append(0)
    if(i[0].isalpha() and '.' in i):
        st_alpha_dot.append(1)
    else:
        st_alpha_dot.append(0)
    if(i.endswith('day')):
        e_day.append(1)
    else:
        e_day.append(0)
#print('\'')

In [50]:
df['hyphen'] = hyphen
df['all_cap'] = all_cap
df['sp_char'] = sp_char
df['cap_dot'] = cap_dot
df['number'] = number
df['st_alpha_dot'] = st_alpha_dot
df['e_day'] = e_day

### converting dataframe to Data Matrix 'X' and target column vector 'y' 

In [51]:
data = np.array(df)

x = data[:,2:]
x.shape
y = df['tag']
y.shape
x.shape

(211727, 33)

### Adding/Appending prev and next context : appending feature vect 

In [52]:
no_cols = x.shape[1] * 5
features = np.zeros((211730,no_cols))

b = no_cols / 5
b = int(b)

In [53]:
# taking prev and next words as feature vectors : appending the stored 
# feature vectors of the same

for i in range(x.shape[0]):
    if(i==0):
        features[i][b * 2: b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]
    
    elif(i == 1):
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 4:b* 5] = x[i+2]
        
    elif(i == x.shape[0]-1):
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]

    elif(i == x.shape[0]-2):
        features[i][b* 3:b* 4] = x[i+1]
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
    
    else:
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]

In [54]:
features[features.shape[0]]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 6., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 6., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0.])

In [56]:
feature_vect = features[:-3]
feature_vect.shape

(211727, 165)

### Importing packages for running softmax model 

In [61]:
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

### Using label encoder to transform target label column to numerical encoding

In [62]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
Y = le.transform(y)

### Storing this mapping of tags to numerical numbers : we ll use this later in viterbi algorithm

In [63]:
pair = dict()
for i in range(Y.shape[0]):
    pair[y[i]] = Y[i]
    
pickle.dump(pair, open('pair_unigrams.pkl', 'wb'))

### Training softmax model 

In [64]:
softmax_2 = LogisticRegression( multi_class = 'multinomial', solver = 'sag')
softmax_2.fit(feature_vect,Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [68]:
# y_pred = softmax_2.predict(X_test)
# accuracy_score(y_pred,y_test)

### Storing Trained softmax model

In [65]:
pickle.dump(softmax_2, open('softmax.pkl', 'wb'))